In [2]:
# Import the standard modules
import pathlib

# Import spiceypy
import spiceypy

# Load necessary SPICE kernels
spiceypy.furnsh('kernel_meta.txt')

# Import the installed modules
import numpy as np

# Import matplotlib for plotting
from matplotlib import pyplot as plt